Ранее решалась задача определения рейтинга статьи на Хабре. Мы столкнулись с проблемой предсказания рейтинга для еще неопубликованных на сайте публикаций, так как использовали в моделях признаки, которые есть только в опубликованных на сайте статьях (количество просмотров, количество закладок, количество комментариев и т.д.).

В рамках прошлой задачи уже была осуществлена предобработка текстов статей (удаление лишних символов, лемматизация). Сейчас перейдем к задаче **предсказания хабов по тексту**.

Автор: Карнакова Ксения

# Hubs prediction

In [1]:
import random
import pandas as pd
import numpy as np
import pickle
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
import nltk
import warnings
warnings.filterwarnings("ignore")

In [107]:
df = pd.read_csv('Habr_lemmatized.csv')

In [108]:
df.head()

,id,is_corporative,posttype,title,tags,time_published,bookmarks,comments_count,views,votes,...,rating,preprocess_text,hubs,hubs_additional,year,month,day,time,weekday,lemmatized_text
0,310000,1,0,Как мы неделю чинили compaction в Cassandra,"cassandra,nosql,troubleshooting",2016-09-17 06:11:00+00:00,45,13,11917,41,...,0.0,Основным хранилищем метрик у нас является cas...,"sys_admin,dwh,devops",okmeter,2016,9,17,06:11:00,5,основной хранилище метрика являться cassandra ...
1,310002,1,0,Войны гипервизоров: To be continued,"облачные вычисления,виртуализация,хостинг,гипе...",2016-09-15 07:56:08+00:00,19,23,7342,0,...,62.3,"Войны гипервизоров — дело непрерывное, и, нав...","it-infrastructure,virtualization,cloud_computing","cloud4y,hosting",2016,9,15,07:56:08,3,война гипервизор дело непрерывный наверное дей...
2,310006,0,0,AdBlock Plus совершил новый виток в истории «п...,"adblock plus,блокировка рекламы,медийная рекла...",2016-09-14 14:26:49+00:00,35,83,44608,36,...,0.0,Один из крупнейших в мире блокировщиков рекла...,"web_monetization,mobile_monetization,display_a...",NaN,2016,9,14,14:26:49,2,крупный мир блокировщик реклама adblock plus п...
3,310008,1,0,Миф о незрелости мобильных NFC технологий,"Gemalto,nfc,мобильные приложения,мобильный пла...",2016-09-15 08:02:54+00:00,12,8,6343,9,...,0.0,Современные NFC-технологии в их нынешнем виде...,"infosecurity,pay_system,ecommerce_development",gemaltorussia,2016,9,15,08:02:54,3,современный nfc технология нынешний вид появля...
4,310014,1,0,OpenJDK: Project Panama,"joker2016,jokerconf,java,jvm,panama",2016-09-15 06:30:51+00:00,50,12,14040,34,...,0.0,Два года назад в OpenJDK был создан новый про...,"programming,java",jugru,2016,9,15,06:30:51,3,год назад openjdk создавать новый проект кодов...


Удалим лишние столбцы, оставив только айди публикации, заголовок, дату публикации, теги, хабы и лемматизированный текст публикации.

In [109]:
habr_df = df[['id', 'title', 'time_published', 'tags', 'hubs', 'lemmatized_text']]
habr_df.head()

,id,title,time_published,tags,hubs,lemmatized_text
0,310000,Как мы неделю чинили compaction в Cassandra,2016-09-17 06:11:00+00:00,"cassandra,nosql,troubleshooting","sys_admin,dwh,devops",основной хранилище метрика являться cassandra ...
1,310002,Войны гипервизоров: To be continued,2016-09-15 07:56:08+00:00,"облачные вычисления,виртуализация,хостинг,гипе...","it-infrastructure,virtualization,cloud_computing",война гипервизор дело непрерывный наверное дей...
2,310006,AdBlock Plus совершил новый виток в истории «п...,2016-09-14 14:26:49+00:00,"adblock plus,блокировка рекламы,медийная рекла...","web_monetization,mobile_monetization,display_a...",крупный мир блокировщик реклама adblock plus п...
3,310008,Миф о незрелости мобильных NFC технологий,2016-09-15 08:02:54+00:00,"Gemalto,nfc,мобильные приложения,мобильный пла...","infosecurity,pay_system,ecommerce_development",современный nfc технология нынешний вид появля...
4,310014,OpenJDK: Project Panama,2016-09-15 06:30:51+00:00,"joker2016,jokerconf,java,jvm,panama","programming,java",год назад openjdk создавать новый проект кодов...


Так как мы используем только профильные хабы (непрофильные хабы, например, относящиеся к компаниям (gemaltorussia, okmeter и т.д.) не учитываем), то есть статьи без хабов, удалим им. Получается, что в датасете оставили только статьи с профильными хабами.

In [3]:
habr_df = habr_df[habr_df.hubs.notna()]

Добавим немного красоты: из строк с тегами и хабами сделаем списки.

In [174]:
def str_to_list(x):
    if x is not np.nan:
        if ',' in x:
            return [j for j in x.split(',')]
        else:
            return [x]

In [176]:
habr_df.tags = habr_df.tags.map(lambda a: a.split(','))
habr_df.hubs = habr_df.hubs.apply(str_to_list)

In [177]:
habr_df.head()

,id,title,time_published,tags,hubs,lemmatized_text
0,310000,Как мы неделю чинили compaction в Cassandra,2016-09-17 06:11:00+00:00,"[cassandra, nosql, troubleshooting]","[sys_admin, dwh, devops]",основной хранилище метрика являться cassandra ...
1,310002,Войны гипервизоров: To be continued,2016-09-15 07:56:08+00:00,"[облачные вычисления, виртуализация, хостинг, ...","[it-infrastructure, virtualization, cloud_comp...",война гипервизор дело непрерывный наверное дей...
2,310006,AdBlock Plus совершил новый виток в истории «п...,2016-09-14 14:26:49+00:00,"[adblock plus, блокировка рекламы, медийная ре...","[web_monetization, mobile_monetization, displa...",крупный мир блокировщик реклама adblock plus п...
3,310008,Миф о незрелости мобильных NFC технологий,2016-09-15 08:02:54+00:00,"[Gemalto, nfc, мобильные приложения, мобильный...","[infosecurity, pay_system, ecommerce_development]",современный nfc технология нынешний вид появля...
4,310014,OpenJDK: Project Panama,2016-09-15 06:30:51+00:00,"[joker2016, jokerconf, java, jvm, panama]","[programming, java]",год назад openjdk создавать новый проект кодов...


Сохраним получившийся датасет.

In [178]:
#habr_df.to_parquet('data/habrdf.parquet', index=False)

In [2]:
habr_df = pd.read_parquet('data/habrdf.parquet')

Разбиваю датасет на трейн, валидацию и тест.

In [3]:
df_train, df_valtest = train_test_split(habr_df, test_size=0.3, random_state=42)
df_val, df_test = train_test_split(df_valtest, test_size=0.5, random_state=42)

In [180]:
#df_train.to_parquet('data/df_train.parquet', index=False)
#df_test.to_parquet('data/df_test.parquet', index=False)
#df_val.to_parquet('data/df_val.parquet', index=False)

In [35]:
df_train.shape, df_test.shape, df_val.shape

((80528, 6), (17256, 6), (17256, 6))

In [8]:
# количество упоминаний хабов на трейне
pd.Series([i for j in df_train['hubs'].values for i in j]).value_counts()

infosecurity         9505
programming          9057
it-infrastructure    5129
machine_learning     4575
webdev               4343
                     ... 
silverlight             1
coffeescript            1
fido                    1
twisted                 1
typo3                   1
Name: count, Length: 321, dtype: int64

In [7]:
# 10 самых популярных хабов на трейне
pd.Series([i for j in df_train['hubs'].values for i in j]).value_counts()[:10]

infosecurity         9505
programming          9057
it-infrastructure    5129
machine_learning     4575
webdev               4343
javascript           3955
python               3816
sys_admin            3732
hr_management        3476
open_source          3201
Name: count, dtype: int64

In [32]:
pd.Series([len(h) for h in habr_df.hubs]).describe()

count    115040.000000
mean          2.237813
std           1.151461
min           1.000000
25%           1.000000
50%           2.000000
75%           3.000000
max           5.000000
dtype: float64

Самое большое количество упомянутых хабов в одной публикации: 5 

Чаще всего в одной публикации указывают хабов: 2 

## Обучение модели

Планируется применить следующие модели:

`Logistic Regression`, `Linear Support Vector Machine`, `Multinomial Naive Bayes`, `Random Forest`, `Catboost`

У текстов статей при помощи регулярных выражений в ноутбуке `EDA/preprocessing_data.ipynb` уже были удалены знаки препинания, цифры и другие неинформативные символы – остались только слова, приведенные к нижнему регистру. После этого, используя `pymystem3`, к текстам статей применялась лемматизация, а теперь для получившихся лемматизированных текстов произведем токенизацию текста по словам.

### TF-IDF

In [4]:
"""%%time
STOPWORDS = nltk.corpus.stopwords.words("russian")
tfidfvectorizer = TfidfVectorizer(analyzer='word', min_df=20, stop_words=STOPWORDS)
df_train['tfidf'] = [i for i in tfidfvectorizer.fit_transform(df_train['lemmatized_text'].values).toarray()]
df_val['tfidf'] = [i for i in tfidfvectorizer.transform(df_val['lemmatized_text'].values).toarray()]"""

CPU times: user 1min 24s, sys: 14.6 s, total: 1min 38s
Wall time: 1min 43s


(min_df=20 так как при меньшем значении падает ядро)

In [36]:
%%time
STOPWORDS = nltk.corpus.stopwords.words("russian")
tfidfvectorizer = TfidfVectorizer(analyzer='word', min_df=20, stop_words=STOPWORDS)
x_train_tfidf = tfidfvectorizer.fit_transform(df_train['lemmatized_text'].values)
x_val_tfidf = tfidfvectorizer.transform(df_val['lemmatized_text'].values)
x_test_tfidf = tfidfvectorizer.transform(df_test['lemmatized_text'].values)

CPU times: user 1min 32s, sys: 5.37 s, total: 1min 38s
Wall time: 1min 43s


In [37]:
x_train_tfidf.shape, x_val_tfidf.shape, x_test_tfidf.shape

((80528, 50606), (17256, 50606), (17256, 50606))

In [22]:
'''with open('data/tfidfvectorizer_new.pkl', 'wb') as file:
    pickle.dump(tfidfvectorizer, file)'''
with open('data/tfidfvectorizer_new.pkl', 'rb') as f:
    tfidfvectorizer = pickle.load(f)

In [7]:
len_vec = x_train_tfidf.shape[1]
print(f'Длина вектора: {len_vec}')

Длина вектора: 50606


In [4]:
hubs_count = pd.Series([i for j in df_train['hubs'].values for i in j]).value_counts()
hubs_dict = dict(zip(hubs_count.index, range(len(hubs_count.index))))
dict_hubs = {j: i for i, j in hubs_dict.items()}

In [20]:
print(f'У нас собраны тексты из {len(hubs_dict)} хаба')

У нас собраны тексты из 321 хаба


Для начала **оставим первый хаб в каждой статье** исходя из предположения, что он - самый главный и наиболее точно описывающий тематику статьи (Multiclass classification). Позже перейдем к классификации хабов с несколькими выходами (Multiclass-multioutput classification).

In [5]:
# пока оставим первый хаб 
y_train = df_train.hubs.map(lambda a: a[0])
y_val = df_val.hubs.map(lambda a: a[0])
y_test = df_test.hubs.map(lambda a: a[0])

### LogisticRegression

Первая модель для обучения - логистическая регрессия.

**Так как задача многоклассовая с несбалансированными классами, то будут использовать следующие метрики:**

`F1-macro` – вычисляет F1-меру для каждого класса, а потом возвращает среднее арифметическое по всем классам, 

`F1-weighted` – вычисляет F1-меру для каждого класса, а потом возвращает среднее арифметическое по всем классам с учетом доли объектов каждого класса в наборе данных.

Разница между метриками `F1-macro` и `F1-weighted` будет показывать, насколько хорошо распознаются классы с небольшим количеством объектов.


`accuracy` и `F1-micro` также считаем для модели, но планируем оценивать качество по `F1-macro` и `F1-weighted`.


In [18]:
logit = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter = 1000, random_state=42, n_jobs=-1)

In [22]:
logit = logit.fit(x_train_tfidf, y_train)


In [184]:
y_pred = logit.predict(x_val_tfidf)
print('accuracy %s' % accuracy_score(y_val, y_pred))
print('f1-score (macro) %s' % f1_score(y_val, y_pred, average='macro'))
print('f1-score (micro) %s' % f1_score(y_val, y_pred, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_val, y_pred, average='weighted'))

accuracy 0.47821047751506723
f1-score (macro) 0.15717769722846486
f1-score (micro) 0.47821047751506723
f1-score (weighted) 0.4436455288546236


In [51]:
print(classification_report(y_val, y_pred, zero_division = 0, target_names=np.unique(y_val)))

                             precision    recall  f1-score   support

                         1C       0.00      0.00      0.00        14
                3d_graphics       0.63      0.41      0.50        82
                       AJAX       0.00      0.00      0.00         2
                  Atlassian       0.00      0.00      0.00         6
                     Hadoop       0.00      0.00      0.00         1
                       IPFS       0.00      0.00      0.00         1
               Sailfish_dev       0.00      0.00      0.00         1
              accessibility       0.00      0.00      0.00        10
                      agile       0.17      0.04      0.06        27
                 algorithms       0.34      0.23      0.28       180
            analysis_design       0.33      0.35      0.34       265
                android_dev       0.41      0.30      0.35       126
                    angular       0.00      0.00      0.00         9
                  animation      

Кажется, что малочисленные классы предсказываются плохо. 

Метрики первой обученной логистической регрессии:
`F1-macro= 0.16` и `F1-weighted=0.44`, `accuracy=0.48`

Посмотрим на конкретном тексте из валидационной выборки, как работает обученная модель:

In [81]:
df_val.lemmatized_text.iloc[130]

'умный разработчик любить работать умный код открывать качественный исходник привлекать талант иллюстрация kevin знать проект facebook открытый исходный код вопрос задавать инженер джеймс пирс бывший директор программа открытый исходный код facebook изучать причина почему приходить компания согласно презентация reilly open source convention треть инженер facebook знать программа приход компания половина сказать это положительно повлиять решение работать facebook facebook одинокий независимо размер компания открывать исходник хороший способ привлекать хороший инженер проанализировать ведущий американский технологический стартапов количество подавать заявка соискание вакансия angellist весь история обнаруживать половина размещать проект open source грамотно применять открытый исходник привлечение инженер целый искусство недостаточно просто опубликовывать репозиторий github надеяться хороший приносить польза рекрутинг следовать подходить каждый проект open source позиция маркетолог писать

In [80]:
test_predicting = tfidfvectorizer.transform([df_val.lemmatized_text.iloc[130]])

In [120]:
print(f'Предсказанный хаб: {logit.predict(test_predicting)}')

Предсказанный хаб: ['open_source']


In [122]:
print(f'Спиок реально указанных хабов: {df_val.hubs.iloc[130]}')

Спиок реально указанных хабов: ['open_source', 'github', 'hr_management']


Еще раз на другом тексте:

In [123]:
df_val.lemmatized_text.iloc[5000]

'прототип робот дуняша становиться первый миссис пермь диана габдуллина апрель год пермский компания промобот представлять кибер кафетерий робот дуняша база робот компаньон robo подвижный лицо голова шея функциональный рука который помогать устройство взаимодействовать собеседник жестикулировать наливать кофе брать предмет выполнять различный движение робот кассир дуняша предлагать купить мороженое газировка кофе время ожидание заказ робот способный поддерживать разговор обсуждать новость предлагать сделать совместный селфи первый рабочий кафетерий дуняша появляться центральный парка пермь нижний новгород май год интерьер кибер кафетерий комплект поставка кибер кафе входить фризер приготовление мягкий мороженое аппарат приготовление газированный напиток кофемашина модуль оснащать роботизированный рука манипулятор также терминал самообслуживание гиперреалистичный робот кассир коммуникация клиент оформление заказ заявлять комплектующий кибер кафе преимущественно российский разработчик ра

In [124]:
test_predicting = tfidfvectorizer.transform([df_val.lemmatized_text.iloc[5000]])

In [125]:
print(f'Предсказанный хаб: {logit.predict(test_predicting)}')

Предсказанный хаб: ['robo_dev']


In [127]:
print(f'Спиок реально указанных хабов: {df_val.hubs.iloc[5000]}')

Спиок реально указанных хабов: ['robo_dev']


In [105]:
pickle.dump(logit, open('logit1.pkl', 'wb'))

Еще раз обучим логистическую регрессию увеличив гиперпараметр `C=1` до `С=2`.

In [134]:
lr = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
lr.fit(x_train_tfidf, y_train)

LogisticRegression(C=2, max_iter=1000, n_jobs=-1)

In [135]:
pickle.dump(lr, open('logit2.pkl', 'wb'))

In [183]:
y_pred = lr.predict(x_val_tfidf)
print('accuracy %s' % accuracy_score(y_val, y_pred))
print('f1-score (macro) %s' % f1_score(y_val, y_pred, average='macro'))
print('f1-score (micro) %s' % f1_score(y_val, y_pred, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_val, y_pred, average='weighted'))

accuracy 0.49038015762633286
f1-score (macro) 0.19108309709914695
f1-score (micro) 0.49038015762633286
f1-score (weighted) 0.4626421264878426


In [144]:
print(classification_report(y_val, y_pred, zero_division=0, target_names=np.unique(y_val)))

                             precision    recall  f1-score   support

                         1C       1.00      0.07      0.13        14
                3d_graphics       0.64      0.45      0.53        82
                       AJAX       0.00      0.00      0.00         2
                  Atlassian       0.00      0.00      0.00         6
                     Hadoop       0.00      0.00      0.00         1
                       IPFS       0.00      0.00      0.00         1
               Sailfish_dev       0.00      0.00      0.00         1
              accessibility       0.00      0.00      0.00        10
                      agile       0.14      0.04      0.06        27
                 algorithms       0.35      0.25      0.29       180
            analysis_design       0.34      0.35      0.34       265
                android_dev       0.44      0.34      0.38       126
                    angular       0.00      0.00      0.00         9
                  animation      


Метрики второй логистической регресии обученной логистической регрессии с `C=2` улучшились:

`F1-macro=0.19`, `F1-weighted=0.46`, `accuracy=0.49`

Также попробую применить модель на конкретном тексте из валидационной выборки.

In [152]:
df_val.lemmatized_text.iloc[4000]

'привет звать олег аккаунт менеджер сегодня рассказывать медийный кампания узнавать медийный кампания директ инструмент особенность отличаться графический объявление рся специалист работать яндекс директ вплотную пользоваться инструмент поиск реклама поиск рся рекламный сеть яндекс реклама сайт партнер яндекс ресурс ретаргетинг возвращение клиент побывать сайт это относительно простой понятный инструмент результат который легко предсказывать измерять медийный кампания вопрос сложный непонятный особенно начинающий специалист разбираться инструмент понимать помощь показываться больший число пользователь значительно усиливать свой рекламный кампания перебивать конкурент особенность медийный кампания директ классификация яндекс медийный кампания это имиджевый баннерный реклама аукционный продукт директ большой выбор настройка таргетинг охват нужный аудитория нацеливание ключевой фраза профиль аудитория оплата показ определение выводить ряд важный особенность медийный кампания яндекс отличи

In [153]:
test_predicting = tfidfvectorizer.transform([df_val.lemmatized_text.iloc[4000]])

In [154]:
print(f'Предсказанный хаб: {lr.predict(test_predicting)}')
print(f'Спиок реально указанных хабов: {df_val.hubs.iloc[4000]}')

Предсказанный хаб: ['internetmarketing']
Спиок реально указанных хабов: ['internetmarketing', 'display_adv', 'context']


**Вывод:** 

В логистической регрессии, в целом, крупные хабы предсказываются очень хорошо, благодаря большому количеству наблюдений и ключевым словам, характерным для конкретного хаба. Малочисленные хабы предсказываются намного хуже, т.к. или очень мало наблюдений, или в статьях мало специальной характерной для хаба лексики.

Каждая модель считалась ~2-2.5 часа, можно попробовать подобрать гридсерчем гиперпараметры, чтобы улучшить метрики, но у меня не получилось дождаться результата за приемлемое время.

### Support Vector Machine

Реализация метода опорных векторов с линейным ядром. 

In [41]:
%%time
svc = LinearSVC(class_weight='balanced')
svc.fit(x_train_tfidf, y_train)

/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


CPU times: user 4min 24s, sys: 2.66 s, total: 4min 27s
Wall time: 4min 30s


LinearSVC(class_weight='balanced')

In [42]:
y_pred = svc.predict(x_val_tfidf)
print('accuracy %s' % accuracy_score(y_val, y_pred))
print('f1-score (macro) %s' % f1_score(y_val, y_pred, average='macro'))
print('f1-score (micro) %s' % f1_score(y_val, y_pred, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_val, y_pred, average='weighted'))

accuracy 0.43237134909596664
f1-score (macro) 0.2561536588535086
f1-score (micro) 0.43237134909596664
f1-score (weighted) 0.43531834456910007


In [44]:
print(classification_report(y_val, y_pred, zero_division=0))

                             precision    recall  f1-score   support

                         1C       0.10      0.14      0.11        14
                3d_graphics       0.47      0.67      0.56        82
                       AJAX       0.00      0.00      0.00         2
                  Atlassian       0.28      0.83      0.42         6
                     Hadoop       0.00      0.00      0.00         1
                       IPFS       0.00      0.00      0.00         1
               Sailfish_dev       0.00      0.00      0.00         1
                       UEFI       0.00      0.00      0.00         0
              accessibility       0.27      0.40      0.32        10
                      agile       0.18      0.44      0.26        27
                 algorithms       0.32      0.31      0.31       180
            analysis_design       0.43      0.27      0.33       265
                android_dev       0.36      0.52      0.43       126
                    angular      

In [45]:
%%time
svc = LinearSVC()
svc.fit(x_train_tfidf, y_train)

/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


CPU times: user 4min 11s, sys: 2.31 s, total: 4min 14s
Wall time: 4min 25s


LinearSVC()

In [46]:
y_pred = svc.predict(x_val_tfidf)
print('accuracy %s' % accuracy_score(y_val, y_pred))
print('f1-score (macro) %s' % f1_score(y_val, y_pred, average='macro'))
print('f1-score (micro) %s' % f1_score(y_val, y_pred, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_val, y_pred, average='weighted'))

accuracy 0.48962679647658786
f1-score (macro) 0.24481143190339674
f1-score (micro) 0.48962679647658786
f1-score (weighted) 0.4696772291333459


In [47]:
print(classification_report(y_val, y_pred, zero_division=0))

                             precision    recall  f1-score   support

                         1C       0.33      0.07      0.12        14
                3d_graphics       0.56      0.49      0.52        82
                       AJAX       0.00      0.00      0.00         2
                  Atlassian       0.50      0.33      0.40         6
                     Hadoop       0.00      0.00      0.00         1
                       IPFS       0.00      0.00      0.00         1
               Sailfish_dev       0.00      0.00      0.00         1
                       UEFI       0.00      0.00      0.00         0
              accessibility       0.33      0.10      0.15        10
                      agile       0.15      0.07      0.10        27
                 algorithms       0.30      0.28      0.29       180
            analysis_design       0.34      0.32      0.33       265
                android_dev       0.38      0.32      0.34       126
                    angular      

Можно попробовать подобрать гридсерчем гиперпараметры.

In [15]:
%%time

parameters = {
    'C':[0.1, 1, 2],
    'multi_class': ['ovr', 'crammer_singer'],
}

svc = LinearSVC(random_state=42, max_iter=500)
gsc = GridSearchCV(svc, parameters, verbose=3, cv=3, scoring='f1_micro', n_jobs=-1)
gsc.fit(x_train_tfidf, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/usr/local/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` wi

[CV 1/3] END ............C=0.1, multi_class=ovr;, score=0.490 total time= 4.1min
[CV 1/3] END ..............C=1, multi_class=ovr;, score=0.480 total time= 4.6min
[CV 2/3] END ..............C=1, multi_class=ovr;, score=0.479 total time= 5.7min
[CV 3/3] END ..............C=1, multi_class=ovr;, score=0.479 total time= 5.8min
[CV 1/3] END ...C=1, multi_class=crammer_singer;, score=0.481 total time=54.8min
[CV 2/3] END ...C=2, multi_class=crammer_singer;, score=0.459 total time=77.6min


/usr/local/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/3] END .C=0.1, multi_class=crammer_singer;, score=0.477 total time=47.5min
[CV 1/3] END ..............C=2, multi_class=ovr;, score=0.466 total time= 6.3min
[CV 2/3] END ..............C=2, multi_class=ovr;, score=0.465 total time= 6.8min
[CV 3/3] END ..............C=2, multi_class=ovr;, score=0.466 total time= 6.9min
[CV 1/3] END ..C=2, multi_class=crammer_singer;, score=0.461 total time=154.7min


/usr/local/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 3/3] END ............C=0.1, multi_class=ovr;, score=0.492 total time= 4.0min
[CV 2/3] END .C=0.1, multi_class=crammer_singer;, score=0.477 total time=34.7min
[CV 2/3] END ...C=1, multi_class=crammer_singer;, score=0.476 total time=58.3min
[CV 3/3] END ..C=2, multi_class=crammer_singer;, score=0.458 total time=139.1min


/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


CPU times: user 4min 9s, sys: 21.7 s, total: 4min 30s
Wall time: 4h 5min 1s


GridSearchCV(cv=3, estimator=LinearSVC(max_iter=500, random_state=42),
             n_jobs=-1,
             param_grid={'C': [0.1, 1, 2],
                         'multi_class': ['ovr', 'crammer_singer']},
             scoring='f1_micro', verbose=3)

In [16]:
print(gsc.best_params_)

{'C': 0.1, 'multi_class': 'ovr'}


In [17]:
bestlinearSVC = gsc.best_estimator_
bestlinearSVC.fit(x_train_tfidf, y_train)
y_pred3 = bestlinearSVC.predict(x_val_tfidf)
print('accuracy %s' % accuracy_score(y_val, y_pred3))
print('f1-score (macro) %s' % f1_score(y_val, y_pred3, average='macro'))
print('f1-score (micro) %s' % f1_score(y_val, y_pred3, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_val, y_pred3, average='weighted'))

/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


[CV 2/3] END ............C=0.1, multi_class=ovr;, score=0.488 total time= 4.0min
[CV 3/3] END .C=0.1, multi_class=crammer_singer;, score=0.478 total time=41.2min
[CV 3/3] END ..C=1, multi_class=crammer_singer;, score=0.477 total time=195.9min
accuracy 0.4987830319888734
f1-score (macro) 0.2024638547642925
f1-score (micro) 0.4987830319888734
f1-score (weighted) 0.4639349535492136


In [18]:
pickle.dump(bestlinearSVC, open('linearSVC.pkl', 'wb'))

Лучший результат при подобранных гридсерчем гиперпараметрах (`C=0.1`, `multi_class='ovr'`): `F1-macro=0.20`,`F1-weighted=0.46`, `accuracy=0.50`. 

In [19]:
%%time
svc = LinearSVC(C=0.01, multi_class='ovr')
svc.fit(x_train_tfidf, y_train)
y_pred4 = svc.predict(x_val_tfidf)
print('accuracy %s' % accuracy_score(y_val, y_pred4))
print('f1-score (macro) %s' % f1_score(y_val, y_pred4, average='macro'))
print('f1-score (micro) %s' % f1_score(y_val, y_pred4, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_val, y_pred4, average='weighted'))

/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


accuracy 0.443266110338433
f1-score (macro) 0.11541303485074349
f1-score (micro) 0.443266110338433
f1-score (weighted) 0.3842216253610764
CPU times: user 4min 8s, sys: 2.96 s, total: 4min 11s
Wall time: 4min 20s


Уменьшение с `С=0.1` до `C=0.01` ухудшило метрики: `F1-macro=0.12`, `F1-weighted=0.38`, `accuracy=0.44`. 

In [27]:
pickle.dump(svc, open('linearSVC_2.pkl', 'wb'))

**Вывод:**

В `LinearSVC` ощутимо быстрее работает дефолтная мультиклассовая стратегия «One-vs-rest» (`multi_class='ovr'`) по сравнению с `multi_class='crammer_singer'`.

При указании `class_weight='balanced'`, как рекомендуется для несбалансированых классов, немного повысилcя `F1-macro`, остальные метрики у  аналогичной модели без указания этого параметра оказались лучше.

**Метрики метода опорных векторов с линейным ядром** с гиперпараметрами, подобранными гридсерчем, **получились немного выше**, чем у логистической регрессии: `F1-macro=0.20`, `F1-weighted=0.46`, `accuracy=0.50` (у лучшей логистической регрессии: `F1-macro=0.19`, `F1-weighted=0.46`, `accuracy=0.49`), при этом **модель обучается очень быстро** (логрег - несколько часов, метод опорных векторов - несколько минут).


### Multinomial Naive Bayes

Перейдем к наивному байесовскому классификатору:

In [24]:
%%time
mnb = MultinomialNB()
mnb.fit(x_train_tfidf, y_train)

CPU times: user 9.55 s, sys: 540 ms, total: 10.1 s
Wall time: 10.8 s


MultinomialNB()

In [28]:
y_pred5 = mnb.predict(x_val_tfidf)
print('accuracy %s' % accuracy_score(y_val, y_pred5))
print('f1-score (macro) %s' % f1_score(y_val, y_pred5, average='macro'))
print('f1-score (micro) %s' % f1_score(y_val, y_pred5, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_val, y_pred5, average='weighted'))

accuracy 0.2706884561891516
f1-score (macro) 0.017860102173864054
f1-score (micro) 0.2706884561891516
f1-score (weighted) 0.17445510643886558


Можно попробовать подобрать гридсерчем гиперпараметры.

In [38]:
%%time

parameters = {'alpha': [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100]}

mnb = MultinomialNB()
gsmnb = GridSearchCV(mnb, parameters, verbose=3, cv=3, scoring='f1_micro', n_jobs=-1)
gsmnb.fit(x_train_tfidf, y_train)

Fitting 3 folds for each of 7 candidates, totalling 21 fits
CPU times: user 10.8 s, sys: 1.63 s, total: 12.4 s
Wall time: 2min 19s


GridSearchCV(cv=3, estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.1, 1, 10, 100]},
             scoring='f1_micro', verbose=3)

In [37]:
print(gsmnb.best_params_)

{'alpha': 0.001}


In [35]:
bestMNB = gsmnb.best_estimator_
bestMNB.fit(x_train_tfidf, y_train)
y_pred6 = bestMNB.predict(x_val_tfidf)
print('accuracy %s' % accuracy_score(y_val, y_pred6))
print('f1-score (macro) %s' % f1_score(y_val, y_pred6, average='macro'))
print('f1-score (micro) %s' % f1_score(y_val, y_pred6, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_val, y_pred6, average='weighted'))

accuracy 0.44025266573945293
f1-score (macro) 0.2053206078614365
f1-score (micro) 0.44025266573945293
f1-score (weighted) 0.42061951692454164


In [39]:
pickle.dump(bestMNB, open('MultinomialNB.pkl', 'wb'))

**Вывод:** 

Очень быстро, но метрики ниже чем у логрега и метода опорных векторов: `F1-macro=0.22`, `F1-weighted=0.42`, `accuracy=0.44`.

## Multiclass-multioutput classification: OneVsRestClassifier(LogisticRegression())

Перейдем к классификации хабов с несколькими выходами (`Multiclass-multioutput classification`). До этого мы выделяли из списка хабов только один (первый), теперь оставим все и попробуем предсказывать их.

Подготовим хабы к класификации:


In [30]:
y_train = df_train.hubs
y_val = df_val.hubs
y_test = df_test.hubs

In [34]:
mlb = MultiLabelBinarizer(classes=sorted(hubs_count.keys()))

y_train = mlb.fit_transform(y_train)
y_val = mlb.fit_transform(y_val)

In [36]:
'''with open('data/MultiLabelBinarizer.pkl', 'wb') as file:
    pickle.dump(mlb, file)'''
with open('data/MultiLabelBinarizer.pkl', 'rb') as file:
    mlb = pickle.load(file)

Обучим дефолтную модель:

In [62]:
%%time

clf = OneVsRestClassifier(LogisticRegression())
clf.fit(x_train_tfidf, y_train)

/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

CPU times: user 42min 35s, sys: 6min 55s, total: 49min 31s
Wall time: 25min 58s


OneVsRestClassifier(estimator=LogisticRegression())

In [63]:
pickle.dump(clf, open('OneVsRestClassifierLogReg.pkl', 'wb'))

In [70]:
y_pred8 = clf.predict(x_val_tfidf)
print('accuracy %s' % accuracy_score(y_val, y_pred8))
print('f1-score (macro) %s' % f1_score(y_val, y_pred8, zero_division=0, average='macro'))
print('f1-score (micro) %s' % f1_score(y_val, y_pred8, zero_division=0, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_val, y_pred8, zero_division=0, average='weighted'))

accuracy 0.15229485396383866
f1-score (macro) 0.19087615121916823
f1-score (micro) 0.4161592550002828
f1-score (weighted) 0.3851780155112786


Подберем гридсерчем гиперпараметры.

In [ ]:
%%time

parameters = {
    'C':[0.1, 1, 10],
    'penalty'; ['l1','l2'],
}

svc = clf()
gsc = GridSearchCV(svc, parameters, verbose=3, cv=3, scoring='f1_micro', n_jobs=-1)
gsc.fit(x_train_tfidf, y_train)

Обучим модель с лучшими гиперпараметрами:

In [73]:
%%time

clf1 = OneVsRestClassifier(LogisticRegression(penalty='l2',C=10, max_iter=1000))
clf1.fit(x_train_tfidf, y_train)

CPU times: user 1h 33min 37s, sys: 15min 13s, total: 1h 48min 51s
Wall time: 55min 39s


OneVsRestClassifier(estimator=LogisticRegression(C=10, max_iter=1000))

In [74]:
pickle.dump(clf1, open('OneVsRestClassifierLogReg_l2_C10.pkl', 'wb'))

In [75]:
y_pred9 = clf1.predict(x_val_tfidf)
print('accuracy %s' % accuracy_score(y_val, y_pred9))
print('f1-score (macro) %s' % f1_score(y_val, y_pred9, zero_division=0, average='macro'))
print('f1-score (micro) %s' % f1_score(y_val, y_pred9, zero_division=0, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_val, y_pred9, zero_division=0, average='weighted'))

accuracy 0.1898470097357441
f1-score (macro) 0.3405405933182099
f1-score (micro) 0.5025125628140703
f1-score (weighted) 0.4822929189164698


У данной модели сильно упала точноть: `accuracy=0.19` (в прошлых экспериментах доходила до `accuracy=0.50`)

In [12]:
with open('OneVsRestClassifierLogReg_l2_C10.pkl', 'rb') as f:
    clf1 = pickle.load(f)

Посмотрим как предсказываются хабы на одном конкретном тексте.

In [18]:
df_test.lemmatized_text.iloc[10]

'спрашивать прожженный системный администратор использовать realtime конфигурация asterisk вероятность ответ отрицательный качество обоснование скоро услышать недоступность источник данные телефония становиться неработоспособный интересно узнавать обходить это ограничение читать далеко тяжелый неказистый жизнь простой программист предупреждение осторожность использование описывать решение данный модификация коробка позволять использовать backend основа curl использование backend требоваться сделать соответствующий доработка протестировать участок код который использоваться наш конфигурация часть изменение сделать прицел будущее полноценный проверяться использование production окружение рекомендоваться выполнять тестирование отладочный сборка включать мониторинг утечка ресурс например valgrind постановка задача алгоритм обработка вызов data abbr диалплан диалплан модифицироваться достаточно редко делать получение информация внешний источник данные мера необходимость реальный время data 

Реальные хабы этой статьи:

In [25]:
df_test.hubs.iloc[10]

['hi', 'asterisk', 'c']

В векторизованном виде:

In [37]:
mlb.fit_transform([df_test.hubs.iloc[10]])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [23]:
test_predicting = tfidfvectorizer.transform([df_test.lemmatized_text.iloc[10]])

Вектор предсказаний хабов:

In [24]:
clf1.predict(test_predicting)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Модель не предсказала ни один из трех хабов. Кажется, что сейчас лучше сосредоточиться на предыдущей идее - обучаться только на одном самом важном хабе и предсказывать один хаб, а не сочетание нескольких.

### TF-IDF (новые параметры)

Ранее при `tf-idf` векторизации был установлен параметр min_df=20 (так как при меньшем значении падало ядро), то есть мы игнорировали слова, которые встречаются меньше двадцати раз. Попрбуем снизить min_df до 12. При этом уберем слова, которые встречаются практически в каждом тексте (например, max_df=0.95).

In [58]:
%%time
STOPWORDS = nltk.corpus.stopwords.words("russian")
tfidfvectorizer = TfidfVectorizer(analyzer='word', min_df=12, max_df=0.95, stop_words=STOPWORDS)
x_train_tfidf = tfidfvectorizer.fit_transform(df_train['lemmatized_text'].values)
x_val_tfidf = tfidfvectorizer.transform(df_val['lemmatized_text'].values)
x_test_tfidf = tfidfvectorizer.transform(df_test['lemmatized_text'].values)

CPU times: user 1min 33s, sys: 5.03 s, total: 1min 38s
Wall time: 1min 43s


In [59]:
'''with open('data/tfidfvectorizer_updated.pkl', 'wb') as file:
    pickle.dump(tfidfvectorizer, file)'''

In [6]:
with open('data/tfidfvectorizer_updated.pkl', 'rb') as f:
    tfidfvectorizer = pickle.load(f)

In [7]:
#x_train_tfidf = tfidfvectorizer.fit_transform(df_train['lemmatized_text'].values)
x_train_tfidf = tfidfvectorizer.transform(df_train['lemmatized_text'].values)
x_val_tfidf = tfidfvectorizer.transform(df_val['lemmatized_text'].values)
x_test_tfidf = tfidfvectorizer.transform(df_test['lemmatized_text'].values)

In [10]:
len_vec = x_train_tfidf.shape[1]
print(f'Длина нового вектора слов: {len_vec} (Длина предыдущего вектора: 50606)')

Длина нового вектора слов: 67809 (Длина предыдущего вектора: 50606)


Лучшие метрики в прошлый раз получились у `логистической регрессии` и `метода опорных векторов с линейным ядром`. Запустим гридсерч для текстов, векторизированных tf-idf с новыми параметрами. Затем сравним изменились ли метрики.

### Support Vector Machine¶

In [61]:
%%time

parameters = {
    'C':[0.05, 0.1, 0.5, 1],
    'multi_class': ['ovr'],
}

svc = LinearSVC(random_state=42, max_iter=500)
gsc = GridSearchCV(svc, parameters, verbose=3, cv=3, scoring='f1_weighted', n_jobs=-1)
gsc.fit(x_train_tfidf, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: Futu

[CV 1/3] END ............C=0.1, multi_class=ovr;, score=0.454 total time= 5.6min
[CV 2/3] END ............C=0.1, multi_class=ovr;, score=0.454 total time= 5.5min
[CV 3/3] END ............C=0.5, multi_class=ovr;, score=0.469 total time= 6.9min
CPU times: user 3min 55s, sys: 4.4 s, total: 3min 59s
Wall time: 23min 20s


GridSearchCV(cv=3, estimator=LinearSVC(max_iter=500, random_state=42),
             n_jobs=-1,
             param_grid={'C': [0.05, 0.1, 0.5, 1], 'multi_class': ['ovr']},
             scoring='f1_weighted', verbose=3)

In [62]:
print(gsc.best_params_)
bestlinearSVC = gsc.best_estimator_
bestlinearSVC.fit(x_train_tfidf, y_train)

{'C': 0.5, 'multi_class': 'ovr'}
[CV 2/3] END ...........C=0.05, multi_class=ovr;, score=0.437 total time= 5.7min
[CV 3/3] END ............C=0.1, multi_class=ovr;, score=0.457 total time= 5.6min
[CV 1/3] END ..............C=1, multi_class=ovr;, score=0.462 total time= 7.6min
[CV 1/3] END ...........C=0.05, multi_class=ovr;, score=0.437 total time= 5.8min
[CV 2/3] END ............C=0.5, multi_class=ovr;, score=0.468 total time= 6.2min
[CV 2/3] END ..............C=1, multi_class=ovr;, score=0.462 total time= 7.3min
[CV 3/3] END ...........C=0.05, multi_class=ovr;, score=0.436 total time= 5.7min
[CV 1/3] END ............C=0.5, multi_class=ovr;, score=0.468 total time= 6.3min
[CV 3/3] END ..............C=1, multi_class=ovr;, score=0.461 total time= 7.3min


LinearSVC(C=0.5, max_iter=500, random_state=42)

In [63]:
pickle.dump(bestlinearSVC, open('linearSVC_new.pkl', 'wb'))

In [65]:
y_pred2 = bestlinearSVC.predict(x_val_tfidf)
print('accuracy %s' % accuracy_score(y_val, y_pred2))
print('f1-score (macro) %s' % f1_score(y_val, y_pred2, average='macro'))
print('f1-score (micro) %s' % f1_score(y_val, y_pred2, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_val, y_pred2, average='weighted'))

accuracy 0.5009851645804357
f1-score (macro) 0.24642570750349482
f1-score (micro) 0.5009851645804357
f1-score (weighted) 0.47765906594017044


**Вывод:**

После векторизации TF-IDF с `min_df=12`, `max_df=0.95` метрики метода опорных векторов с линейным ядром с гиперпараметрами, подобранными гридсерчем, получились немного выше (`F1-macro=0.25`, `F1-weighted=0.48`, `accuracy=0.50`), чем после применения этой модели на текстах после векторизации TF-IDF с `min_df=20` (`F1-macro=0.20`, `F1-weighted=0.46`, `accuracy=0.50`).


### Logistic Regression

In [11]:
lr = LogisticRegression(C=2, max_iter=1000, n_jobs=-1)
lr.fit(x_train_tfidf, y_train)

LogisticRegression(C=2, max_iter=1000, n_jobs=-1)

In [12]:
pickle.dump(lr, open('logit3.pkl', 'wb'))

In [14]:
y_pred2 = lr.predict(x_val_tfidf)
print('accuracy %s' % accuracy_score(y_val, y_pred2))
print('f1-score (macro) %s' % f1_score(y_val, y_pred2, average='macro'))
print('f1-score (micro) %s' % f1_score(y_val, y_pred2, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_val, y_pred2, average='weighted'))

accuracy 0.4898006490496059
f1-score (macro) 0.18938110287116913
f1-score (micro) 0.4898006490496059
f1-score (weighted) 0.4618292749488603


In [16]:
print(classification_report(y_val, y_pred2, zero_division = 0))

                             precision    recall  f1-score   support

                         1C       1.00      0.07      0.13        14
                3d_graphics       0.66      0.45      0.54        82
                       AJAX       0.00      0.00      0.00         2
                  Atlassian       0.00      0.00      0.00         6
                     Hadoop       0.00      0.00      0.00         1
                       IPFS       0.00      0.00      0.00         1
               Sailfish_dev       0.00      0.00      0.00         1
              accessibility       0.00      0.00      0.00        10
                      agile       0.12      0.04      0.06        27
                 algorithms       0.34      0.24      0.28       180
            analysis_design       0.34      0.36      0.35       265
                android_dev       0.45      0.35      0.39       126
                    angular       0.00      0.00      0.00         9
                  animation      

**Вывод:**

После векторизации TF-IDF с `min_df=12`, `max_df=0.95` метрики логистической регрессии получились такими же, как и после применения этой модели на текстах после векторизации TF-IDF с `min_df=20`: `F1-macro=0.19`, `F1-weighted=0.46`, `accuracy=0.49`.


### Random Forest

Попробуем применить случайный лес.

In [19]:
rfc = RandomForestClassifier(n_estimators=300, max_depth=15, random_state=42)
rfc.fit(x_train_tfidf, y_train)

RandomForestClassifier(max_depth=15, n_estimators=300, random_state=42)

In [20]:
y_pred = rfc.predict(x_test_tfidf)
print('accuracy %s' % accuracy_score(y_test, y_pred))
print('f1-score (macro) %s' % f1_score(y_test, y_pred, average='macro'))
print('f1-score (micro) %s' % f1_score(y_test, y_pred, average='micro'))
print('f1-score (weighted) %s' % f1_score(y_test, y_pred, average='weighted'))

accuracy 0.23794622160407974
f1-score (macro) 0.015995270104949645
f1-score (micro) 0.23794622160407974
f1-score (weighted) 0.14702807871557397


In [21]:
print(classification_report(y_test, y_pred, zero_division = 0))

                             precision    recall  f1-score   support

                         1C       0.00      0.00      0.00        13
                3d_graphics       0.00      0.00      0.00        93
                       AJAX       0.00      0.00      0.00         7
                  Atlassian       0.00      0.00      0.00        10
                       IPFS       0.00      0.00      0.00         1
               Sailfish_dev       0.00      0.00      0.00         2
                    Xamarin       0.00      0.00      0.00         1
              accessibility       0.00      0.00      0.00         3
                      agile       0.00      0.00      0.00        21
                 algorithms       0.00      0.00      0.00       192
            analysis_design       0.00      0.00      0.00       272
                android_dev       0.00      0.00      0.00       137
                    angular       0.00      0.00      0.00        13
                  animation      

In [22]:
pickle.dump(rfc, open('randforest.pkl', 'wb'))

**Вывод:**

Случайный лес требует больших вычислительных и временных ресурсов. Одна модель с заранее заданными гиперпараметрами в виде 300 деревьев и глубины 15 считалась ~3 часа. По результатам получилось, что предсказываются только совсем крупные хабы, метрики низкие: `F1-macro=0.02`, `F1-weighted=0.15`, `accuracy=0.24`. Чтобы улучшить метрики, нужно работать над гиперпараметрами, но это не позволяет сделать ограничение по памяти.
